# Simulation Behaviors with LLM based on the context

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import re
import json

In [2]:
chat_model = ChatOllama(model='phi3:instruct')

In [3]:
class AnswerOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        choice_pattern = re.compile(r'```(?:json)?(.*?)```', re.DOTALL)
        choice_match = choice_pattern.search(text)
        choice = ""
        try:
            choice = choice_match.group(1)
            return json.loads(choice.strip()), text.strip()
        except Exception as e:
            return {},text.strip()

In [4]:


def get_llm_choice(node_type, profile, top_k, old_context, new_options):
    system_propmt = '''
    You are an expert analyst of social networks, given the context of connect propobilities with the similar node in the past, evaluate the possibilities of new options.
    '''
    user_prompt = '''
    Given a {node_type} Node with profile:
    {profile}
    And the context of {top_k} options and their probabilities a similar node connected in the past:
    {old_context}
    The new options are:
    {new_options}
    Is there any possibility for each new option the given node would connect to? 
    Note:
    - Answer with a list of 'Yes' or 'No' for each new option in json forma, using the index as the key for each option.
    - Wrap the final answer in triple backticks (```json ```) to indicate a json block.
    - Follw with the reasons for your choice after the json block.
    Answer Format Example:
    ```json
    {{
        "0": "Yes",
        "1": "No"
    }}
    [Reasons for the choices]
    ```
    '''

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",system_propmt),
            ("human", user_prompt),
        ]
    )
    choice_response = (
        RunnablePassthrough.assign(
            node_type=lambda _: node_type,
            profile=lambda _: profile,
            top_k=lambda _: top_k,
            old_context=lambda _: old_context,
            new_options=lambda _: new_options
        )
        | prompt
        | chat_model.bind()
        | AnswerOutputParser()
    )
    choice, response = choice_response.invoke({
        "node_type": node_type,
        "profile": profile,
        "top_k": top_k,
        "old_context": old_context,
        "new_context": new_options
    })
    return choice, response



# One Step Simulation

Actors ---> Actors 

When a new person come to the community who are more likely to connect with him?

In [5]:
from choice_model import ChoiceModel
import networkx as nx
model = ChoiceModel()

In [6]:
new_profile = {
'age': '18-35',
'gender': 'Female',
'marrige': 'Yes',
'withkids': 'No',
'student': 'Yes',
'workstatue': 'Freelancer',
'residentinneighbor': 'No',
'educationlevel': 'Undergraduate'
}
choices,probabilities = model.link_prediction(profile=new_profile, k1=10, k2=5, node_type='Actors', choice_type='Actors', top_k=3)
options = [ c['properties'] for c in choices]
options_probabilities = [ p for p in probabilities]
old_context = {f"option:{o}, probability:{p}" for o,p in zip(options,options_probabilities)}

In [7]:

new_options = [{'age': '80', 'gender': 'Female', 'marrige': 'No', 'withkids': 'No', 'student': 'No', 'workstatue': 'Employed', 'residentinneighbor': 'Yes', 'educationlevel': 'Highschool'},{'age': '18', 'gender': 'Male', 'marrige': 'Yes', 'withkids': 'No', 'student': 'No', 'workstatue': 'Freelancer', 'residentinneighbor': 'No', 'educationlevel': 'Undergraduate'}]

In [8]:
choice, response = get_llm_choice(node_type='Actors', profile=new_profile, top_k=3, old_context=old_context, new_options=new_options)

In [9]:
print(choice)

{'0': 'No', '1': 'Yes'}


In [10]:
print(response)

```json
{
    "0": "No",
    "1": "Yes"
}
```
Explanation: Option 0 is less likely to connect because it represents an age significantly different from the original node, which could indicate a very distinct lifestyle and opportunities that might not align well with those of similar past connections. Furthermore, being '80' would be quite unusual in this context compared to other nodes listed as having individuals within their 20s or early 30s age range.

Option 1 is more probable since the given profile closely matches one of the previously connected options (option:{'age': '18-35', 'gender': 'Female', 'marrige': 'Yes', 'withkids': 'No', 'student': 'Yes', 'workstatue': 'Freelancer', 'residentinneighbor': 'No', 'educationlevel': 'Undergraduate'}, probability:0.3385717503580057). The age, gender, marital status, and education level are aligned with the original node's profile which makes it a reasonable match based on past connections that have similar attributes.


Event ---> Actors

When a new Event happend who are likely to be attracted to the event

In [11]:
period = 1
new_event = {'eventtopic': 'Environment And Sustainability'}
model.set_period(period)
choices,probabilities = model.link_prediction(profile=new_event, k1=10, k2=5, node_type='Event', choice_type='Actors',top_k=3)
options = [ c['properties'] for c in choices]
options_probabilities = [ p for p in probabilities]
old_context = {f"option:{o}, probability:{p}" for o,p in zip(options,options_probabilities)}


In [12]:
new_options = [{'age': '80', 'gender': 'Female', 'marrige': 'No', 'withkids': 'No', 'student': 'No', 'workstatue': 'Employed', 'residentinneighbor': 'Yes', 'educationlevel': 'Highschool'},{'age': '18', 'gender': 'Male', 'marrige': 'Yes', 'withkids': 'No', 'student': 'No', 'workstatue': 'Freelancer', 'residentinneighbor': 'No', 'educationlevel': 'Undergraduate'}]

In [13]:
choice, response = get_llm_choice(node_type='Event', profile=new_profile, top_k=3, old_context=old_context, new_options=new_options)

In [14]:
choice

{'0': 'No', '1': 'Yes'}

In [15]:
print(response)

```json
{
    "0": "No",
    "1": "Yes"
}
```
The first new option has a very low likelihood of connecting based on age, gender, and marital status differences. The 80-year-old female with no children is less likely to connect considering the typical demographics represented in social networks where younger individuals tend to be more active participants due to their lifestyle choices and availability (e.g., students or working professionals).

The second option shows a high potential for connection since it matches several key attributes such as age, education level, work status, marital status, presence of children which is similar to the original profile but differs only in gender that might not significantly impact social network connections based on shared interests and values. Therefore: 
- "0": The significant difference in age reduces connectivity potential since there's less commonality for interaction between such disparate ages within typical online communities or event netw

Actors ---> Event

what Event whould a person like comparing to other events

In [16]:
new_profile = {
'age': '18-35',
'gender': 'Female',
'marrige': 'Yes',
'withkids': 'No',
'student': 'Yes',
'workstatue': 'Freelancer',
'residentinneighbor': 'No',
'educationlevel': 'Undergraduate'
}
choices,probabilities = model.link_prediction(profile=new_profile, k1=10, k2=5, node_type='Actors', choice_type='Event', top_k=3)
options = [ c['properties'] for c in choices]
options_probabilities = [ p for p in probabilities]
old_context = {f"option:{o}, probability:{p}" for o,p in zip(options,options_probabilities)}

In [18]:
new_options = [{'eventtopic': 'Culture And Arts'},{'eventtopic': 'Environment And Sustainability'}]

In [19]:
choice, response = get_llm_choice(node_type='Event', profile=new_profile, top_k=3, old_context=old_context, new_options=new_options)

In [20]:
print(choice)

{'0': 'Yes', '1': 'No'}


In [21]:
print(response)

```json
{
        "0": "Yes",
        "1": "No"
}
```
Answer: The given profile suggests that this user is likely interested in 'Culture And Arts' due to her being a student and showing interest in cultural activities. While it does not directly indicate an interest in 'Environment And Sustainability', the fact she has no children could imply more freedom or time for environmental causes, though indirectly suggesting such topics might have less connection than with culture-related events specifically designed for students like her who are likely to be exposed to various cultural activities. Nonetheless, since there is a direct past connection already established and given that 'Environment And Sustainability' isn’t in the list of previous options connected but still holds relevance due to broader societal trends (even if not directly indicated by this profile), it could be plausible for her to connect with such events, albeit less likely than culture-related ones. Hence 'Yes' is given 